<a href="https://colab.research.google.com/github/manashprotim27/LDA_class/blob/main/LDA_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install lda


In [2]:
import logging
import re
import lda
import nltk
import numpy as np
import pandas as pd
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [46]:
class ldaMod(BaseEstimator):  
    
    def __init__(self, num_topics=6, num_iterations=100, random_state=None, clean_text=True, vectorizer=None):
        self.num_topics = num_topics
        self.num_iterations = num_iterations
 
        self.random_state = random_state
        self.lda_model = lda.LDA(n_topics=self.num_topics, n_iter=self.num_iterations, random_state=self.random_state)
        

    def clean_texts(input_str):

        input_str =[re.sub(r'[-.!\n]', ' ',sent) for sent in input_str]
        input_str = [re.sub(r'[^\s^\w]', ' ',sent)for sent in input_str]
        input_str = [re.sub(r'[0-9]', ' ',sent)for sent in input_str]
        return (input_str) 
   
    def listToString(s): 
        strng = ""  
        for ele in s: 
            strng += ele    
        return strng     

    def preprocess(input_str):
    
        input_str = input_str.lower()
        document_words = nltk.word_tokenize(input_str)
        lemmatizer = WordNetLemmatizer()
        stems = list()
        for word in document_words:
            stems.append(lemmatizer.lemmatize(word))
        stop = stopwords.words('english')  
        stop.extend(['the'])
        stems = filter(lambda x: x not in stop, stems)
        output = u' '.join(stems)
        return output;
        
    

    def vector(norm_corpus):
        norm_corpus=norm_corpus.split()
        cv = CountVectorizer(min_df=0., max_df=1.)
        cv_matrix = cv.fit_transform(norm_corpus)
        cv_matrix = cv_matrix.toarray()
        vocab = cv.get_feature_names()
        return  cv_matrix,vocab
    
    
    def topic_model_coherence_generator(topic_num_start=2,
                                    topic_num_end=6,
                                    norm_corpus='',
                                    cv_matrix='',
                                    cv=''):
        norm_corpus_tokens = [doc.split() for doc in norm_corpus]
        models = []
        coherence_scores = []

        for i in range(topic_num_start, topic_num_end):
            print(i)
            new_lda = lda.LDA(i)
            new_lda.fit_transform(cv_matrix)
            cur_coherence_score = metric_coherence_gensim(
                measure='c_v',
                top_n=5,
                topic_word_distrib=cur_lda.components_,
                dtm=cv.fit_transform(norm_corpus),
            vocab=np.array(cv.get_feature_names()),
            texts=norm_corpus_tokens)
            models.append(new_lda)
            coherence_scores.append(np.mean(cur_coherence_score))
        return models, coherence_scores


In [4]:
from sklearn.datasets import fetch_20newsgroups


In [5]:
Texts_Data=fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [6]:
Texts_Data=Texts_Data.data

In [10]:
Text_words = list(ldaMod.clean_texts(Texts_Data))

In [26]:
Text_words

['I was wondering if anyone out there could enlighten me on this car I saw the other day  It was a   door sports car  looked to be from the late   s  early   s  It was called a Bricklin  The doors were really small  In addition  the front bumper was separate from the rest of the body  This is  all I know  If anyone can tellme a model name  engine specs  years of production  where this car is made  history  or whatever info you have on this funky looking car  please e mail ',
 'A fair number of brave souls who upgraded their SI clock oscillator have shared their experiences for this poll  Please send a brief message detailing your experiences with the procedure  Top speed attained  CPU rated speed  add on cards and adapters  heat sinks  hour of usage per day  floppy disk functionality with     and     m floppies are especially requested   I will be summarizing in the next two days  so please add to the network knowledge base if you have done the clock upgrade and haven t answered this p

In [11]:
 a= ldaMod.listToString(Text_words)

In [12]:
b=ldaMod.preprocess(a)

In [43]:
b

'wa wondering anyone could enlighten car saw day wa door sport car looked late early wa called bricklin door really small addition front bumper wa separate rest body know anyone tellme model name engine spec year production car made history whatever info funky looking car please e mail fair number brave soul upgraded si clock oscillator shared experience poll please send brief message detailing experience procedure top speed attained cpu rated speed add card adapter heat sink hour usage per day floppy disk functionality floppy especially requested summarizing next two day please add network knowledge base done clock upgrade answered poll thanks well folk mac plus finally gave ghost weekend starting life k way back sooo market new machine bit sooner intended looking picking powerbook maybe bunch question hopefully somebody answer doe anybody know dirt next round powerbook introduction expected heard c wa supposed make appearence summer heard anymore since access macleak wa wondering any

In [13]:
b= b[:20000]

In [14]:
d,e=ldaMod.vector(b)

In [42]:
d

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [80]:
import numpy as np
np.max(d)

1

In [20]:
obj=ldaMod(num_topics=3,num_iterations=100,random_state=None,clean_text=True,vectorizer=None)
           

In [21]:
Model=obj.lda_model

In [22]:
f=Model.fit_transform(d)

INFO:lda:n_documents: 3034
INFO:lda:vocab_size: 1501
INFO:lda:n_words: 2971
INFO:lda:n_topics: 3
INFO:lda:n_iter: 100
INFO:lda:<0> log likelihood: -30226
INFO:lda:<10> log likelihood: -26716
INFO:lda:<20> log likelihood: -26717
INFO:lda:<30> log likelihood: -26679
INFO:lda:<40> log likelihood: -26691
INFO:lda:<50> log likelihood: -26696
INFO:lda:<60> log likelihood: -26707
INFO:lda:<70> log likelihood: -26722
INFO:lda:<80> log likelihood: -26725
INFO:lda:<90> log likelihood: -26716
INFO:lda:<99> log likelihood: -26692


In [35]:
pip install tmtoolkit

     |████████████████████████████████| 7.1 MB 24.4 MB/s 
     |████████████████████████████████| 11.5 MB 12 kB/s 
     |████████████████████████████████| 10.4 MB 41.1 MB/s 
     |████████████████████████████████| 96 kB 5.3 MB/s 
     |████████████████████████████████| 25.9 MB 952 bytes/s 
     |████████████████████████████████| 1.0 MB 37.3 MB/s 
  Created wheel for globre: filename=globre-0.1.5-py3-none-any.whl size=19545 sha256=3efe1a879d8792faa0d3557e80c3948330e7432c6ef96cd7520a4dde1f55e7ec
  Stored in directory: /root/.cache/pip/wheels/40/95/37/5303ce04fce53b6e64ed74a3f0a0e9ea11c348cac0c5c42a76
Successfully built globre
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
 

In [36]:
import tmtoolkit
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim

In [38]:
ts = 2
te = 10
cv = CountVectorizer(min_df=0., max_df=1.)
models, coherence_scores =ldaMod.topic_model_coherence_generator(
    ts, te, norm_corpus=b.split(), cv=cv, cv_matrix=d)

INFO:lda:n_documents: 3034
INFO:lda:vocab_size: 1501
INFO:lda:n_words: 2971
INFO:lda:n_topics: 2
INFO:lda:n_iter: 2000
INFO:lda:<0> log likelihood: -28942
INFO:lda:<10> log likelihood: -26635
INFO:lda:<20> log likelihood: -26602


2


INFO:lda:<30> log likelihood: -26624
INFO:lda:<40> log likelihood: -26573
INFO:lda:<50> log likelihood: -26610
INFO:lda:<60> log likelihood: -26588
INFO:lda:<70> log likelihood: -26602
INFO:lda:<80> log likelihood: -26571
INFO:lda:<90> log likelihood: -26585
INFO:lda:<100> log likelihood: -26610
INFO:lda:<110> log likelihood: -26616
INFO:lda:<120> log likelihood: -26611
INFO:lda:<130> log likelihood: -26639
INFO:lda:<140> log likelihood: -26605
INFO:lda:<150> log likelihood: -26617
INFO:lda:<160> log likelihood: -26612
INFO:lda:<170> log likelihood: -26599
INFO:lda:<180> log likelihood: -26600
INFO:lda:<190> log likelihood: -26603
INFO:lda:<200> log likelihood: -26596
INFO:lda:<210> log likelihood: -26576
INFO:lda:<220> log likelihood: -26609
INFO:lda:<230> log likelihood: -26614
INFO:lda:<240> log likelihood: -26643
INFO:lda:<250> log likelihood: -26609
INFO:lda:<260> log likelihood: -26614
INFO:lda:<270> log likelihood: -26597
INFO:lda:<280> log likelihood: -26569
INFO:lda:<290> log 

3


INFO:lda:<30> log likelihood: -26698
INFO:lda:<40> log likelihood: -26730
INFO:lda:<50> log likelihood: -26725
INFO:lda:<60> log likelihood: -26701
INFO:lda:<70> log likelihood: -26682
INFO:lda:<80> log likelihood: -26705
INFO:lda:<90> log likelihood: -26720
INFO:lda:<100> log likelihood: -26702
INFO:lda:<110> log likelihood: -26714
INFO:lda:<120> log likelihood: -26749
INFO:lda:<130> log likelihood: -26728
INFO:lda:<140> log likelihood: -26757
INFO:lda:<150> log likelihood: -26708
INFO:lda:<160> log likelihood: -26775
INFO:lda:<170> log likelihood: -26684
INFO:lda:<180> log likelihood: -26730
INFO:lda:<190> log likelihood: -26699
INFO:lda:<200> log likelihood: -26670
INFO:lda:<210> log likelihood: -26748
INFO:lda:<220> log likelihood: -26715
INFO:lda:<230> log likelihood: -26712
INFO:lda:<240> log likelihood: -26700
INFO:lda:<250> log likelihood: -26671
INFO:lda:<260> log likelihood: -26676
INFO:lda:<270> log likelihood: -26676
INFO:lda:<280> log likelihood: -26687
INFO:lda:<290> log 

...

INFO:lda:<30> log likelihood: -27089
INFO:lda:<40> log likelihood: -27121
INFO:lda:<50> log likelihood: -27120
INFO:lda:<60> log likelihood: -27089
INFO:lda:<70> log likelihood: -27090
INFO:lda:<80> log likelihood: -27145
INFO:lda:<90> log likelihood: -27110
INFO:lda:<100> log likelihood: -27143
INFO:lda:<110> log likelihood: -27036
INFO:lda:<120> log likelihood: -27137
INFO:lda:<130> log likelihood: -27050
INFO:lda:<140> log likelihood: -27109
INFO:lda:<150> log likelihood: -27141
INFO:lda:<160> log likelihood: -27097
INFO:lda:<170> log likelihood: -27112
INFO:lda:<180> log likelihood: -27136
INFO:lda:<190> log likelihood: -27096
INFO:lda:<200> log likelihood: -27089
INFO:lda:<210> log likelihood: -27065
INFO:lda:<220> log likelihood: -27149
INFO:lda:<230> log likelihood: -27069
INFO:lda:<240> log likelihood: -27037
INFO:lda:<250> log likelihood: -27097
INFO:lda:<260> log likelihood: -27147
INFO:lda:<270> log likelihood: -27086
INFO:lda:<280> log likelihood: -27096
INFO:lda:<290> log 

8


INFO:lda:<30> log likelihood: -27185
INFO:lda:<40> log likelihood: -27172
INFO:lda:<50> log likelihood: -27174
INFO:lda:<60> log likelihood: -27192
INFO:lda:<70> log likelihood: -27162
INFO:lda:<80> log likelihood: -27291
INFO:lda:<90> log likelihood: -27154
INFO:lda:<100> log likelihood: -27164
INFO:lda:<110> log likelihood: -27217
INFO:lda:<120> log likelihood: -27152
INFO:lda:<130> log likelihood: -27264
INFO:lda:<140> log likelihood: -27231
INFO:lda:<150> log likelihood: -27283
INFO:lda:<160> log likelihood: -27249
INFO:lda:<170> log likelihood: -27245
INFO:lda:<180> log likelihood: -27241
INFO:lda:<190> log likelihood: -27239
INFO:lda:<200> log likelihood: -27281
INFO:lda:<210> log likelihood: -27193
INFO:lda:<220> log likelihood: -27158
INFO:lda:<230> log likelihood: -27303
INFO:lda:<240> log likelihood: -27214
INFO:lda:<250> log likelihood: -27226
INFO:lda:<260> log likelihood: -27213
INFO:lda:<270> log likelihood: -27224
INFO:lda:<280> log likelihood: -27171
INFO:lda:<290> log 

9


INFO:lda:<30> log likelihood: -27274
INFO:lda:<40> log likelihood: -27149
INFO:lda:<50> log likelihood: -27277
INFO:lda:<60> log likelihood: -27238
INFO:lda:<70> log likelihood: -27298
INFO:lda:<80> log likelihood: -27282
INFO:lda:<90> log likelihood: -27296
INFO:lda:<100> log likelihood: -27269
INFO:lda:<110> log likelihood: -27253
INFO:lda:<120> log likelihood: -27262
INFO:lda:<130> log likelihood: -27252
INFO:lda:<140> log likelihood: -27206
INFO:lda:<150> log likelihood: -27294
INFO:lda:<160> log likelihood: -27248
INFO:lda:<170> log likelihood: -27328
INFO:lda:<180> log likelihood: -27264
INFO:lda:<190> log likelihood: -27275
INFO:lda:<200> log likelihood: -27319
INFO:lda:<210> log likelihood: -27281
INFO:lda:<220> log likelihood: -27299
INFO:lda:<230> log likelihood: -27289
INFO:lda:<240> log likelihood: -27229
INFO:lda:<250> log likelihood: -27203
INFO:lda:<260> log likelihood: -27325
INFO:lda:<270> log likelihood: -27276
INFO:lda:<280> log likelihood: -27296
INFO:lda:<290> log 

In [39]:
models

In [40]:
coherence_scores

[0.5351961395712106,
 0.5207637325228097,
 0.509113400625661,
 0.5014537035397432,
 0.4942780483597377,
 0.4904881034829871,
 0.48249884896345074,
 0.477397446008299]

In [41]:
bestlda=models[0]

New instances


In [ ]:
input=[ Now  take a good look at at  an tell me man  there is no Christian Devil  There is  is real  is a virus  a meme  infecting and possessing the good people and keep  em from becoming human beings with emphasis on the being  Is not a matter of good people an evil people  is all good people see  but some good people vexed of the Christian Devil  An it can t be burn out or lynch out or rape out  Only wise up let I rise up  Christian Devil is real man  how else can you explain five hundred years of history  even more  Can only be explained by Christians invoke Christian Devil   you keep on knocking but you can t come in  i got to understand you ve been living in sin  but walk right in and sit right down  i ll keep on loving you  i ll play the clown  but bend down low  let i tell you what i know yah  i ve been  buked brothers and i ve been stoned  woe  woe  woe  now i m hung by a tree in the the ganging on a few  woe  woe  woe  it doesn t matter who the man is who lives the life he loves  it doesn t matter what the man does or the honest life he loves  i want somewhere  i want somewhere  hallelujah  hallelujah  somewhere to lay my head  woe is me  only ska beat in  eaven man  stiff necked fools  you think you re cool  to deny me for simplicity  yes you have gone  for so long with your love for vanity now  yes you have got the wrong interpretation mixed up with vain imagination  so take jah sun and jah moon and jah rain and jah stars  and forever yes erase your fantasy  yeah  the lips of the righteous teach many  but fools die for want of wisdom  the rich man s wealth is in his city  the righteous wealth is in his holy place  so take jah sun and jah moon and jah rain and jah stars  and forever yes erase your fantasy  destruction of the poor is in their poverty  destruction of the soul is vanity  yeah  but i don t want to rule ya  i don t want to fool ya  i don t want to school ya  things you  you might never know about  yes you have got the wrong interpretation mixed up with vain  vain imagination  stiff necked fools  you think you re cool  to deny me for  o                           ]

In [45]:
x=ldaMod.clean_texts(input)

TypeError: ignored